### 只是简单的看一下代码，深入点以标题的形式列出，就像这里一样

In [4]:
import time
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# self define
sys.path.append('../../utils/')
import Prepare
import Overview
import Completing
import Correcting
import Modelling

In [9]:
# It takes long to run the cope, so i comment them and upload my result
sale_train = pd.read_csv('../data/sales_train.csv')
test  = pd.read_csv('../data/test.csv.gz')

### 4C中，应该是用一般的3*IQR得到的数据，同一商品价格变动很大，应该先找到其主要影响因素，然后分组检验异常

In [11]:
sale_train[sale_train['item_id'] == 11373][['item_price']].sort_values(['item_price'])

,item_price
2909818,0.908714
2257993,38.500000
2048642,71.000000
1058343,72.200000
2462729,75.454545
1398729,76.000000
2608354,76.083333
2257974,77.500000
2626184,79.677419
2731663,80.000000


In [12]:
sale_train[sale_train['item_id'] == 11365].sort_values(['item_price'])

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1651714,16.05.2014,16,12,11365,124.000000,5.0
2805487,21.08.2015,31,12,11365,170.000000,2.0
1330776,13.01.2014,12,12,11365,180.000000,3.0
1398688,25.02.2014,13,12,11365,194.000000,5.0
661581,05.07.2013,6,12,11365,230.000000,4.0
2731490,30.07.2015,30,12,11365,230.000000,1.0
1398683,06.02.2014,13,12,11365,246.666667,9.0
2626123,21.05.2015,28,12,11365,248.333333,6.0
2608316,16.04.2015,27,12,11365,249.333333,15.0
1058265,29.11.2013,10,12,11365,260.867925,53.0


In [17]:
sale_train['item_cnt_day'][2909818]

4.0

In [13]:
# Correct sale_train values
sale_train['item_price'][2909818] = np.nan
sale_train['item_cnt_day'][2909818] = np.nan
sale_train['item_price'][2909818] = sale_train[(sale_train['shop_id'] ==12) & (sale_train['item_id'] == 11373) & (sale_train['date_block_num'] == 33)]['item_price'].median()
sale_train['item_cnt_day'][2909818] = round(sale_train[(sale_train['shop_id'] ==12) & (sale_train['item_id'] == 11373) & (sale_train['date_block_num'] == 33)]['item_cnt_day'].median())
sale_train['item_price'][885138] = np.nan
sale_train['item_price'][885138] = sale_train[(sale_train['item_id'] == 11365) & (sale_train['shop_id'] ==12) & (sale_train['date_block_num'] == 8)]['item_price'].median()

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

### 不知道为什么要merge？？？估计是只考虑test集中出现的ID，其他的多余的商品就不做考虑了

In [18]:
test_nrow = test.shape[0]
sale_train = sale_train.merge(test[['shop_id']].drop_duplicates(), how = 'inner')
sale_train['date'] = pd.to_datetime(sale_train['date'], format = '%d.%m.%Y')

In [19]:
sale_train.sample(14)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
722893,2014-08-31,19,4,15775,399.0000,1.0
371984,2014-06-27,17,22,4821,299.0000,1.0
714562,2013-12-22,11,4,4029,99.0000,1.0
1279020,2013-12-28,11,31,11030,149.0000,1.0
1049314,2013-11-16,10,10,1202,671.0000,1.0
1194440,2013-01-02,0,31,16152,557.3200,1.0
2362466,2014-09-24,20,49,3346,3199.0000,1.0
2116127,2014-07-08,18,5,19002,229.0000,1.0
2097677,2013-05-10,4,5,6845,299.0000,1.0
343225,2015-09-06,32,19,19985,299.0000,1.0


In [27]:
sale_train.shape

(2413246, 6)

### 不懂为什么要建这个grid？？？

In [20]:
# # 1. Aggregate data ----------------------------------------------------------------
from itertools import product

# # For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sale_train['date_block_num'].unique():
    cur_shops = sale_train[sale_train['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sale_train[sale_train['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

In [38]:
grid[3]

array([[   59,  2730,     3],
       [   59,  2737,     3],
       [   59,  2760,     3],
       ...,
       [   58, 13778,     3],
       [   58, 13991,     3],
       [   58, 14040,     3]], dtype=int32)

In [39]:
# #turn the grid into pandas dataframe
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

In [46]:
grid.sample(15)

,shop_id,item_id,date_block_num
3595595,57,15548,12
7157795,6,228,27
678216,56,20418,2
4541603,47,12679,16
8109676,44,1329,31
7382592,2,18774,28
6826738,58,19812,25
103256,7,824,0
2159767,38,5554,7
5114872,58,5324,18


In [42]:
sale_train['item_cnt_day'].describe()

count    2.413246e+06
mean     1.215979e+00
std      1.542208e+01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+01
Name: item_cnt_day, dtype: float64

In [40]:
# index_cols = ['shop_id', 'item_id', 'date_block_num']
sale_train['item_cnt_day'] = sale_train['item_cnt_day'].clip(0,20)

In [41]:
sale_train['item_cnt_day'].describe()

count    2.413246e+06
mean     1.215979e+00
std      1.543680e+01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.000000e+01
Name: item_cnt_day, dtype: float64

### 月销售也clip(0,20)就有点过了吧，日销售量就有20的了？？？

In [43]:
gb_cnt = sale_train.groupby(index_cols)['item_cnt_day'].agg(['sum']).reset_index().rename(columns = {'sum': 'item_cnt_month'})
gb_cnt['item_cnt_month'] = gb_cnt['item_cnt_month'].clip(0,20).astype(np.int)

In [44]:
gb_cnt['item_cnt_month'].describe()

count    1.323795e+06
mean     2.017895e+00
std      6.708934e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      2.000000e+01
Name: item_cnt_month, dtype: float64

In [54]:
gb_cnt.sample(10)

,shop_id,item_id,date_block_num,item_cnt_month
778608,37,12528,24,1
587728,28,9513,20,1
849027,41,21863,14,4
1000720,47,4357,19,4
863677,42,5636,3,1
590308,28,10498,22,2
863814,42,5657,30,5
120125,6,18005,18,1
59198,5,2956,10,1
733749,35,3476,11,1


### 为什么要downcast？？？

In [47]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int64", "int32"]]

    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int16)

    return df

In [48]:
# #join aggregated data to the grid
train = pd.merge(grid,gb_cnt,how='left',on=index_cols).fillna(0)
train['item_cnt_month'] = train['item_cnt_month'].astype(int)
train = downcast_dtypes(train)

In [50]:
train.sample(10) # 存在大量的0值。

,shop_id,item_id,date_block_num,item_cnt_month
7760337,19,5822,30,14
331732,2,9669,1,0
2293534,4,21546,8,0
679650,56,19299,2,0
1671961,24,8343,6,0
6913926,4,21812,26,0
3189622,14,7709,11,0
1037499,45,20994,3,0
5465190,2,16954,20,0
1157737,3,4473,4,0


In [55]:
# #sort the data
train.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
# print('%0.2f min: Finish joining gb_cnt'%((time.time() - start_time)/60))

In [58]:
# # # Sanity check
print(sale_train['item_cnt_day'].sum())
print(train['item_cnt_month'].sum())  # 因为有clip所以结果会不一样，从这里也可以看出被减掉了多少，所以这样真的可以吗
print(gb_cnt['item_cnt_month'].sum())

2934456.0
2671279
2671279


In [60]:
item = pd.read_csv('../data/items.csv')
train = train.merge(item[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')
test = test.merge(item[['item_id', 'item_category_id']], on = ['item_id'], how = 'left')

In [61]:
train.sample(10)

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id
615042,22,8721,2,1,40
6733915,31,15784,25,7,40
4835396,48,8400,17,1,40
407505,37,8588,1,0,40
5076482,45,19092,18,0,38
5733514,25,12523,21,1,55
5671301,7,22050,21,0,40
4699361,14,17625,17,0,43
6563736,47,13179,24,0,31
2896715,25,15312,10,0,63


### 感觉没必要总和一些类别，像ps2，ps3虽然都是主机，但是销量相差应该是很大的

In [65]:
item_cat = pd.read_csv('../data/item_categories.csv')
l_cat = list(item_cat.item_category_name)
for ind in range(0,1):
    l_cat[ind] = 'PC Headsets / Headphones'
for ind in range(1,8):
    l_cat[ind] = 'Access'
l_cat[8] = 'Tickets (figure)'
l_cat[9] = 'Delivery of goods'
for ind in range(10,18):
    l_cat[ind] = 'Consoles'
for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'
l_cat[25] = 'Accessories for games'
for ind in range(26,28):
    l_cat[ind] = 'phone games'
for ind in range(28,32):
    l_cat[ind] = 'CD games'
for ind in range(32,37):
    l_cat[ind] = 'Card'
for ind in range(37,43):
    l_cat[ind] = 'Movie'
for ind in range(43,55):
    l_cat[ind] = 'Books'
for ind in range(55,61):
    l_cat[ind] = 'Music'
for ind in range(61,73):
    l_cat[ind] = 'Gifts'
for ind in range(73,79):
    l_cat[ind] = 'Soft'
for ind in range(79,81):
    l_cat[ind] = 'Office'
for ind in range(81,83):
    l_cat[ind] = 'Clean'
l_cat[83] = 'Elements of a food'

In [66]:
l_cat

['PC Headsets / Headphones',
 'Access',
 'Access',
 'Access',
 'Access',
 'Access',
 'Access',
 'Access',
 'Tickets (figure)',
 'Delivery of goods',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Consoles Games',
 'Accessories for games',
 'phone games',
 'phone games',
 'CD games',
 'CD games',
 'CD games',
 'CD games',
 'Card',
 'Card',
 'Card',
 'Card',
 'Card',
 'Movie',
 'Movie',
 'Movie',
 'Movie',
 'Movie',
 'Movie',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Books',
 'Music',
 'Music',
 'Music',
 'Music',
 'Music',
 'Music',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Gifts',
 'Soft',
 'Soft',
 'Soft',
 'Soft',
 'Soft',
 'Soft',
 'Office',
 'Office',
 'Clean',
 'Clean',
 'E

In [69]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
item_cat['item_cat_id_fix'] = lb.fit_transform(l_cat)

In [72]:
train = train.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')
test = test.merge(item_cat[['item_cat_id_fix', 'item_category_id']], on = ['item_category_id'], how = 'left')

In [73]:
train.sample(10)

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id,item_cat_id_fix
48789,12,11415,0,0,40,11
7885960,42,21378,30,0,41,11
8513884,41,1519,33,0,31,3
3814323,47,17499,13,0,37,11
2774347,2,9063,10,0,40,11
3982090,24,18596,14,0,40,11
7923081,52,889,30,0,78,15
2567435,15,10331,9,2,40,11
2393363,38,19192,8,0,40,11
5670599,7,19193,21,0,40,11


In [76]:
import gc
del item, item_cat, grid, gb_cnt
gc.collect() #内存清理
# print('%0.2f min: Finish adding item_cat_id_fix'%((time.time() - start_time)/60))

41

In [77]:
# # 2. Add item/shop pair mean-encodings -----------------------------------------
# # For Trainset
# print('%0.2f min: Start adding mean-encoding for item_cnt_month'%((time.time() - start_time)/60))
Target = 'item_cnt_month'
global_mean =  train[Target].mean()
y_tr = train[Target].values

In [82]:
train.describe()

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id,item_cat_id_fix
count,8.598044e+06,8.598044e+06,8.598044e+06,8.598044e+06,8.598044e+06,8.598044e+06
mean,3.088666e+01,1.125405e+04,1.564003e+01,3.106845e-01,4.479533e+01,9.428261e+00
std,2.534912e+02,5.226096e+07,1.261154e+02,1.668830e+01,3.835943e+02,2.254650e+01
min,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.500000e+01,5.864000e+03,7.000000e+00,0.000000e+00,3.700000e+01,7.000000e+00
50%,3.100000e+01,1.134400e+04,1.500000e+01,0.000000e+00,4.000000e+01,1.100000e+01
75%,4.700000e+01,1.654000e+04,2.400000e+01,0.000000e+00,5.500000e+01,1.100000e+01
max,5.900000e+01,2.216900e+04,3.300000e+01,2.000000e+01,8.300000e+01,1.700000e+01


In [86]:
from tqdm import tqdm
SEED = 0
mean_encoded_col = ['shop_id', 'item_id', 'item_category_id', 'item_cat_id_fix']
for col in tqdm(mean_encoded_col):

    col_tr = train[[col] + [Target]]
    corrcoefs = pd.DataFrame(columns = ['Cor'])

    # 3.1.1 Mean encodings - KFold scheme
    from sklearn.model_selection import KFold
    kf = KFold(n_splits = 5, shuffle = False, random_state = SEED)

    col_tr[col + '_cnt_month_mean_Kfold'] = global_mean
    for tr_ind, val_ind in kf.split(col_tr):
        X_tr, X_val = col_tr.iloc[tr_ind], col_tr.iloc[val_ind]
        means = X_val[col].map(X_tr.groupby(col)[Target].mean())
        X_val[col + '_cnt_month_mean_Kfold'] = means
        col_tr.iloc[val_ind] = X_val
        # X_val.head()
    col_tr.fillna(global_mean, inplace = True)
    corrcoefs.loc[col + '_cnt_month_mean_Kfold'] = np.corrcoef(y_tr, col_tr[col + '_cnt_month_mean_Kfold'])[0][1]

    # 3.1.2 Mean encodings - Leave-one-out scheme
    item_id_target_sum = col_tr.groupby(col)[Target].sum()
    item_id_target_count = col_tr.groupby(col)[Target].count()
    col_tr[col + '_cnt_month_sum'] = col_tr[col].map(item_id_target_sum)
    col_tr[col + '_cnt_month_count'] = col_tr[col].map(item_id_target_count)
    col_tr[col + '_target_mean_LOO'] = (col_tr[col + '_cnt_month_sum'] - col_tr[Target]) / (col_tr[col + '_cnt_month_count'] - 1)
    col_tr.fillna(global_mean, inplace = True)
    corrcoefs.loc[col + '_target_mean_LOO'] = np.corrcoef(y_tr, col_tr[col + '_target_mean_LOO'])[0][1]


    # 3.1.3 Mean encodings - Smoothing
    item_id_target_mean = col_tr.groupby(col)[Target].mean()
    item_id_target_count = col_tr.groupby(col)[Target].count()
    col_tr[col + '_cnt_month_mean'] = col_tr[col].map(item_id_target_mean)
    col_tr[col + '_cnt_month_count'] = col_tr[col].map(item_id_target_count)
    alpha = 100
    col_tr[col + '_cnt_month_mean_Smooth'] = (col_tr[col + '_cnt_month_mean'] *  col_tr[col + '_cnt_month_count'] + global_mean * alpha) / (alpha + col_tr[col + '_cnt_month_count'])
    col_tr[col + '_cnt_month_mean_Smooth'].fillna(global_mean, inplace=True)
    corrcoefs.loc[col + '_cnt_month_mean_Smooth'] = np.corrcoef(y_tr, col_tr[col + '_cnt_month_mean_Smooth'])[0][1]


    # 3.1.4 Mean encodings - Expanding mean scheme
    cumsum = col_tr.groupby(col)[Target].cumsum() - col_tr[Target]
    sumcnt = col_tr.groupby(col).cumcount()
    col_tr[col + '_cnt_month_mean_Expanding'] = cumsum / sumcnt
    col_tr[col + '_cnt_month_mean_Expanding'].fillna(global_mean, inplace=True)
    corrcoefs.loc[col + '_cnt_month_mean_Expanding'] = np.corrcoef(y_tr, col_tr[col + '_cnt_month_mean_Expanding'])[0][1]

    train = pd.concat([train, col_tr[corrcoefs['Cor'].idxmax()]], axis = 1)
    print(corrcoefs.sort_values('Cor'))
    #print('%0.2f min: Finish encoding %s'%((time.time() - start_time)/60, col))

# print('%0.2f min: Finish adding mean-encoding'%((time.time() - start_time)/60))


  0%|          | 0/4 [00:00<?, ?it/s]/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

                                       Cor
shop_id_cnt_month_mean_Kfold      0.279501
shop_id_target_mean_LOO           0.288939
shop_id_cnt_month_mean_Smooth     0.296603
shop_id_cnt_month_mean_Expanding  0.296660


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

 50%|█████     | 2/4 [00:31<00:31, 15.64s/it]

                                       Cor
item_id_cnt_month_mean_Kfold      0.381529
item_id_cnt_month_mean_Smooth     0.517359
item_id_target_mean_LOO           0.526035
item_id_cnt_month_mean_Expanding  0.594303


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)

 75%|███████▌  | 3/4 [00:42<00:14, 14.31s/it]

                                                Cor
item_category_id_cnt_month_mean_Kfold      0.357688
item_category_id_target_mean_LOO           0.373443
item_category_id_cnt_month_mean_Smooth     0.373655
item_category_id_cnt_month_mean_Expanding  0.375095


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

100%|██████████| 4/4 [00:55<00:00, 13.95s/it]


                                               Cor
item_cat_id_fix_target_mean_LOO           0.281020
item_cat_id_fix_cnt_month_mean_Kfold      0.285811
item_cat_id_fix_cnt_month_mean_Smooth     0.294306
item_cat_id_fix_cnt_month_mean_Expanding  0.297337


In [87]:
train.sample(10)

,shop_id,item_id,date_block_num,item_cnt_month,item_category_id,item_cat_id_fix,shop_id_target_mean_LOO,shop_id_cnt_month_mean_Expanding,item_id_cnt_month_mean_Expanding,item_category_id_cnt_month_mean_Expanding,item_cat_id_fix_cnt_month_mean_Expanding
2628203,26,14489,9,0,41,11,0.273758,0.298963,0.024000,0.190788,0.234384
1882694,50,6640,6,2,29,3,0.323735,0.409262,0.157025,1.013889,0.996809
961412,38,21016,3,0,61,10,0.228880,0.217788,0.045455,0.135135,0.306312
3791691,44,10113,13,0,40,11,0.195278,0.218967,0.037267,0.257302,0.239467
245208,59,14669,0,0,59,12,0.208932,0.300040,0.218750,0.098128,0.172192
7703461,50,1611,29,0,55,12,0.323744,0.341177,0.124783,0.232395,0.182397
3035499,58,4400,10,0,55,12,0.377654,0.374463,0.136187,0.222442,0.176772
6586493,52,10904,24,0,55,12,0.213532,0.223643,0.317560,0.233142,0.184257
3717146,22,8684,13,0,55,12,0.236462,0.250391,1.294118,0.224817,0.177761
1481401,25,4608,5,1,58,12,0.937805,0.974834,0.113821,0.054731,0.163758


In [89]:
Validation = False
reduce_size = False
num_first_level_models = 3 
# # 2. Feature Engineering -----------------------------------------
# # 2.1 Combine trainset and testset -----------------------------------------
#print('%0.2f min: Start combining data'%((time.time() - start_time)/60))
if Validation == False:
    test['date_block_num'] = 34
    all_data = pd.concat([train, test], axis = 0)
    all_data = all_data.drop(columns = ['ID'])
else:
    all_data = train

del train, test, col_tr
gc.collect()

all_data = downcast_dtypes(all_data)

### train shift并不知道是为了什么？？？

In [90]:
# # 2.2 Creating item/shop pair lags lag-based features ----------------------------
# print('%0.2f min: Start adding lag-based feature'%((time.time() - start_time)/60))
index_cols = ['shop_id', 'item_id', 'item_category_id', 'item_cat_id_fix', 'date_block_num']
cols_to_rename = list(all_data.columns.difference(index_cols))
print(cols_to_rename)
shift_range = [1, 2, 3, 4, 12]

for month_shift in tqdm(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()

    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift

    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift
gc.collect()


  0%|          | 0/5 [00:00<?, ?it/s]

['item_cat_id_fix_cnt_month_mean_Expanding', 'item_category_id_cnt_month_mean_Expanding', 'item_cnt_month', 'item_id_cnt_month_mean_Expanding', 'shop_id_cnt_month_mean_Expanding', 'shop_id_target_mean_LOO']



100%|██████████| 5/5 [01:09<00:00, 13.91s/it]


400

In [91]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8812244 entries, 0 to 8812243
Data columns (total 41 columns):
date_block_num                                      int16
item_cat_id_fix                                     int16
item_cat_id_fix_cnt_month_mean_Expanding            float32
item_category_id                                    int16
item_category_id_cnt_month_mean_Expanding           float32
item_cnt_month                                      float32
item_id                                             int16
item_id_cnt_month_mean_Expanding                    float32
shop_id                                             int16
shop_id_cnt_month_mean_Expanding                    float32
shop_id_target_mean_LOO                             float32
item_cat_id_fix_cnt_month_mean_Expanding_lag_1      float32
item_category_id_cnt_month_mean_Expanding_lag_1     float32
item_cnt_month_lag_1                                float32
item_id_cnt_month_mean_Expanding_lag_1              float

In [95]:
all_data.describe()

,date_block_num,item_cat_id_fix,item_cat_id_fix_cnt_month_mean_Expanding,item_category_id,item_category_id_cnt_month_mean_Expanding,item_cnt_month,item_id,item_id_cnt_month_mean_Expanding,shop_id,shop_id_cnt_month_mean_Expanding,...,item_cnt_month_lag_4,item_id_cnt_month_mean_Expanding_lag_4,shop_id_cnt_month_mean_Expanding_lag_4,shop_id_target_mean_LOO_lag_4,item_cat_id_fix_cnt_month_mean_Expanding_lag_12,item_category_id_cnt_month_mean_Expanding_lag_12,item_cnt_month_lag_12,item_id_cnt_month_mean_Expanding_lag_12,shop_id_cnt_month_mean_Expanding_lag_12,shop_id_target_mean_LOO_lag_12
count,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,...,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06,8.812244e+06
mean,1.608631e+01,9.417880e+00,3.314475e-01,4.483214e+01,3.267387e-01,3.031327e-01,1.124835e+04,4.610252e-01,3.090504e+01,3.147666e-01,...,2.586976e-01,3.626840e-01,1.993027e-01,1.909368e-01,9.906727e-02,1.030028e-01,1.509513e-01,1.940398e-01,9.504217e-02,9.054717e-02
std,1.340237e+02,2.254698e+01,3.768622e-01,3.840935e+02,3.090556e+00,1.646058e+01,4.912518e+07,8.294518e+00,4.012555e+02,1.583183e-01,...,1.575390e+01,9.668254e+00,1.678317e-01,1.515325e-01,2.146278e-01,2.432675e+00,1.300645e+01,8.237782e+00,1.484102e-01,1.312365e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.000000e+00,7.000000e+00,1.812175e-01,3.700000e+01,1.855950e-01,0.000000e+00,5.856000e+03,7.055215e-02,1.500000e+01,1.960607e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.600000e+01,1.100000e+01,2.363316e-01,4.000000e+01,2.304997e-01,0.000000e+00,1.134100e+04,1.643357e-01,3.100000e+01,2.604674e-01,...,0.000000e+00,8.435374e-02,1.887340e-01,1.890072e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.400000e+01,1.100000e+01,2.989794e-01,5.500000e+01,2.656191e-01,0.000000e+00,1.653100e+04,3.962264e-01,4.700000e+01,3.469464e-01,...,0.000000e+00,2.956989e-01,2.866671e-01,2.798978e-01,1.676300e-01,1.165906e-01,0.000000e+00,7.017544e-02,1.415953e-01,1.860294e-01
max,3.400000e+01,1.700000e+01,8.000000e+00,8.300000e+01,2.000000e+01,2.000000e+01,2.216900e+04,2.000000e+01,5.900000e+01,8.400000e+00,...,2.000000e+01,2.000000e+01,8.400000e+00,1.180862e+00,8.000000e+00,2.000000e+01,2.000000e+01,2.000000e+01,8.400000e+00,1.180862e+00


### 为什么要踢掉整年的数据？？？

In [96]:
all_data = all_data[all_data['date_block_num'] >= 12] # Don't use old data from year 2013
lag_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]]
all_data = downcast_dtypes(all_data)
# print('%0.2f min: Finish generating lag features'%((time.time() - start_time)/60))

In [97]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5459310 entries, 3352934 to 8812243
Data columns (total 41 columns):
date_block_num                                      int16
item_cat_id_fix                                     int16
item_cat_id_fix_cnt_month_mean_Expanding            float32
item_category_id                                    int16
item_category_id_cnt_month_mean_Expanding           float32
item_cnt_month                                      float32
item_id                                             int16
item_id_cnt_month_mean_Expanding                    float32
shop_id                                             int16
shop_id_cnt_month_mean_Expanding                    float32
shop_id_target_mean_LOO                             float32
item_cat_id_fix_cnt_month_mean_Expanding_lag_1      float32
item_category_id_cnt_month_mean_Expanding_lag_1     float32
item_cnt_month_lag_1                                float32
item_id_cnt_month_mean_Expanding_lag_1             

### 获取日期数据

In [99]:
# # 2.3 Creating date features --------------------------------------------------------
# print('%0.2f min: Start getting date features'%((time.time() - start_time)/60))

dates_train = sale_train[['date', 'date_block_num']].drop_duplicates()
dates_test = dates_train[dates_train['date_block_num'] == 34-12]
dates_test['date_block_num'] = 34
dates_test['date'] = dates_test['date'] + pd.DateOffset(years=1)
dates_all = pd.concat([dates_train, dates_test])

dates_all['dow'] = dates_all['date'].dt.dayofweek
dates_all['year'] = dates_all['date'].dt.year
dates_all['month'] = dates_all['date'].dt.month
dates_all = pd.get_dummies(dates_all, columns=['dow'])
dow_col = ['dow_' + str(x) for x in range(7)]
date_features = dates_all.groupby(['year', 'month', 'date_block_num'])[dow_col].agg('sum').reset_index()
date_features['days_of_month'] = date_features[dow_col].sum(axis=1)
date_features['year'] = date_features['year'] - 2013

date_features = date_features[['month', 'year', 'days_of_month', 'date_block_num']]
all_data = all_data.merge(date_features, on = 'date_block_num', how = 'left')
date_columns = date_features.columns.difference(set(index_cols))
#print('%0.2f min: Finish getting date features'%((time.time() - start_time)/60))

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [117]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5459310 entries, 0 to 5459309
Data columns (total 44 columns):
date_block_num                                      int16
item_cat_id_fix                                     int16
item_cat_id_fix_cnt_month_mean_Expanding            float32
item_category_id                                    int16
item_category_id_cnt_month_mean_Expanding           float32
item_cnt_month                                      float32
item_id                                             int16
item_id_cnt_month_mean_Expanding                    float32
shop_id                                             int16
shop_id_cnt_month_mean_Expanding                    float32
shop_id_target_mean_LOO                             float32
item_cat_id_fix_cnt_month_mean_Expanding_lag_1      float32
item_category_id_cnt_month_mean_Expanding_lag_1     float32
item_cnt_month_lag_1                                float32
item_id_cnt_month_mean_Expanding_lag_1              float

In [116]:
all_data.drop(['month_x','month_y','year_x','year_y','days_of_month_x','days_of_month_y'], axis = 1, inplace = True)

In [119]:
# # 2.4 Scale feature columns --------------------------------------------------------
from sklearn.preprocessing import StandardScaler
train = all_data[all_data['date_block_num']!= all_data['date_block_num'].max()]
test = all_data[all_data['date_block_num']== all_data['date_block_num'].max()]
sc = StandardScaler()

to_drop_cols = ['date_block_num']
feature_columns = list(set(lag_cols + index_cols + list(date_columns)).difference(to_drop_cols))

In [120]:
feature_columns

['item_cat_id_fix_cnt_month_mean_Expanding_lag_1',
 'item_category_id_cnt_month_mean_Expanding_lag_4',
 'item_category_id_cnt_month_mean_Expanding_lag_12',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_2',
 'item_cnt_month_lag_3',
 'item_id_cnt_month_mean_Expanding_lag_2',
 'item_cat_id_fix',
 'item_id_cnt_month_mean_Expanding_lag_3',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_4',
 'item_cnt_month_lag_1',
 'shop_id_target_mean_LOO_lag_12',
 'year',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_3',
 'item_id_cnt_month_mean_Expanding_lag_4',
 'item_category_id',
 'item_category_id_cnt_month_mean_Expanding_lag_1',
 'shop_id_cnt_month_mean_Expanding_lag_12',
 'shop_id',
 'shop_id_target_mean_LOO_lag_2',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_12',
 'item_id_cnt_month_mean_Expanding_lag_12',
 'item_id',
 'item_cnt_month_lag_4',
 'item_category_id_cnt_month_mean_Expanding_lag_2',
 'shop_id_cnt_month_mean_Expanding_lag_3',
 'days_of_month',
 'shop_id_cnt_month_mean_Expanding_lag_

In [121]:
train[feature_columns] = sc.fit_transform(train[feature_columns])
test[feature_columns] = sc.transform(test[feature_columns])
all_data = pd.concat([train, test], axis = 0)
all_data = downcast_dtypes(all_data)

del train, test, date_features, sale_train
gc.collect()
# print('%0.2f min: Finish scaling features'%((time.time() - start_time)/60))

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

519

In [123]:
all_data.describe()

,date_block_num,item_cat_id_fix,item_cat_id_fix_cnt_month_mean_Expanding,item_category_id,item_category_id_cnt_month_mean_Expanding,item_cnt_month,item_id,item_id_cnt_month_mean_Expanding,shop_id,shop_id_cnt_month_mean_Expanding,...,shop_id_target_mean_LOO_lag_4,item_cat_id_fix_cnt_month_mean_Expanding_lag_12,item_category_id_cnt_month_mean_Expanding_lag_12,item_cnt_month_lag_12,item_id_cnt_month_mean_Expanding_lag_12,shop_id_cnt_month_mean_Expanding_lag_12,shop_id_target_mean_LOO_lag_12,month,year,days_of_month
count,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,...,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06,5.459310e+06
mean,2.252263e+01,-3.721958e-01,3.302184e-01,-1.236124e-01,3.294513e-01,2.866961e-01,-2.330086e-04,4.934899e-01,-8.173595e-02,2.970364e-01,...,-1.366621e+00,-6.758021e-01,-5.981665e-02,-1.456053e-02,-3.301732e-02,-8.363239e-01,-9.104651e-01,-3.965204e-01,-4.825154e+00,-1.770074e+01
std,4.460070e+01,1.186734e-01,4.381337e-01,3.068073e-03,3.560026e+00,1.567833e+01,2.281317e-08,9.844536e+00,2.855652e-03,1.464492e-01,...,6.298985e+00,3.997909e+00,3.926854e-01,6.644414e-02,1.093283e-01,2.819795e+00,5.674870e+00,2.066922e-01,2.720318e+00,9.874893e+00
min,1.200000e+01,-8.744909e-01,0.000000e+00,-2.475718e-01,0.000000e+00,0.000000e+00,-4.657838e-04,0.000000e+00,-1.583438e-01,0.000000e+00,...,-2.775675e+00,-1.354382e+00,-1.201399e-01,-3.062472e-02,-6.647790e-02,-1.692085e+00,-1.818002e+00,-7.619310e-01,-6.358341e+00,-1.943196e+01
25%,1.700000e+01,-4.028895e-01,1.833760e-01,-1.453260e-01,1.836180e-01,0.000000e+00,-3.485712e-04,7.589286e-02,-1.219582e-01,1.897220e-01,...,-2.775675e+00,-1.354382e+00,-1.201399e-01,-3.062472e-02,-6.647790e-02,-1.692085e+00,-1.818002e+00,-6.231819e-01,-6.358341e+00,-1.824772e+01
50%,2.200000e+01,-3.557294e-01,2.305560e-01,-1.370358e-01,2.301700e-01,0.000000e+00,-2.307980e-04,1.849427e-01,-7.517677e-02,2.461601e-01,...,-1.511714e+00,-1.354382e+00,-1.201399e-01,-3.062472e-02,-6.647790e-02,-1.692085e+00,-1.818002e+00,-4.150582e-01,-6.358341e+00,-1.765560e+01
75%,2.800000e+01,-3.085693e-01,3.006537e-01,-9.558482e-02,2.573706e-01,0.000000e+00,-1.240693e-04,4.486405e-01,-4.139015e-02,3.263782e-01,...,-9.323362e-01,-3.519854e-01,-3.873265e-02,-3.062472e-02,-4.321098e-02,-2.915467e-01,-2.173560e-01,-2.069345e-01,-2.909745e+00,-1.765560e+01
max,3.400000e+01,3.644769e-01,6.417709e+00,9.175926e-02,1.830667e+01,2.000000e+01,2.272913e-04,2.000000e+01,-1.020250e-02,1.212305e+00,...,4.867599e+00,3.259345e+01,7.136198e+00,1.244077e+00,2.069849e+00,4.447813e+01,5.514289e+00,3.474674e-01,1.993723e+00,-2.420822e-01


In [124]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5459310 entries, 0 to 5459309
Data columns (total 44 columns):
date_block_num                                      int16
item_cat_id_fix                                     float32
item_cat_id_fix_cnt_month_mean_Expanding            float32
item_category_id                                    float32
item_category_id_cnt_month_mean_Expanding           float32
item_cnt_month                                      float32
item_id                                             float32
item_id_cnt_month_mean_Expanding                    float32
shop_id                                             float32
shop_id_cnt_month_mean_Expanding                    float32
shop_id_target_mean_LOO                             float32
item_cat_id_fix_cnt_month_mean_Expanding_lag_1      float32
item_category_id_cnt_month_mean_Expanding_lag_1     float32
item_cnt_month_lag_1                                float32
item_id_cnt_month_mean_Expanding_lag_1           

In [125]:
# # 3. First-level model ------------------------------------------------------------------
# # Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts
dates = all_data['date_block_num']
last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)
print(feature_columns)

Test `date_block_num` is 34
['item_cat_id_fix_cnt_month_mean_Expanding_lag_1', 'item_category_id_cnt_month_mean_Expanding_lag_4', 'item_category_id_cnt_month_mean_Expanding_lag_12', 'item_cat_id_fix_cnt_month_mean_Expanding_lag_2', 'item_cnt_month_lag_3', 'item_id_cnt_month_mean_Expanding_lag_2', 'item_cat_id_fix', 'item_id_cnt_month_mean_Expanding_lag_3', 'item_cat_id_fix_cnt_month_mean_Expanding_lag_4', 'item_cnt_month_lag_1', 'shop_id_target_mean_LOO_lag_12', 'year', 'item_cat_id_fix_cnt_month_mean_Expanding_lag_3', 'item_id_cnt_month_mean_Expanding_lag_4', 'item_category_id', 'item_category_id_cnt_month_mean_Expanding_lag_1', 'shop_id_cnt_month_mean_Expanding_lag_12', 'shop_id', 'shop_id_target_mean_LOO_lag_2', 'item_cat_id_fix_cnt_month_mean_Expanding_lag_12', 'item_id_cnt_month_mean_Expanding_lag_12', 'item_id', 'item_cnt_month_lag_4', 'item_category_id_cnt_month_mean_Expanding_lag_2', 'shop_id_cnt_month_mean_Expanding_lag_3', 'days_of_month', 'shop_id_cnt_month_mean_Expanding_la

### 这种分层预测的方法也是第一次见，先用原数据训练多个模型，然后用这些模型预测对数据进行预测，然后把多个模型预测的结果做为下一层模型训练的数据，训练模型，并进行最终预测。

In [126]:
import time
num_first_level_models = 3 
# print('%0.2f min: Start training First level models'%((time.time() - start_time)/60))
start_first_level_total = time.perf_counter()

scoringMethod = 'r2'; from sklearn.metrics import mean_squared_error; from math import sqrt


# # Train meta-features M = 15 (12 + 15 = 27)
months_to_generate_meta_features = range(27,last_block +1)
mask = dates.isin(months_to_generate_meta_features)
Target = 'item_cnt_month'
y_all_level2 = all_data[Target][mask].values
X_all_level2 = np.zeros([y_all_level2.shape[0], num_first_level_models])

In [132]:
feature_columns

['item_cat_id_fix_cnt_month_mean_Expanding_lag_1',
 'item_category_id_cnt_month_mean_Expanding_lag_4',
 'item_category_id_cnt_month_mean_Expanding_lag_12',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_2',
 'item_cnt_month_lag_3',
 'item_id_cnt_month_mean_Expanding_lag_2',
 'item_cat_id_fix',
 'item_id_cnt_month_mean_Expanding_lag_3',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_4',
 'item_cnt_month_lag_1',
 'shop_id_target_mean_LOO_lag_12',
 'year',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_3',
 'item_id_cnt_month_mean_Expanding_lag_4',
 'item_category_id',
 'item_category_id_cnt_month_mean_Expanding_lag_1',
 'shop_id_cnt_month_mean_Expanding_lag_12',
 'shop_id',
 'shop_id_target_mean_LOO_lag_2',
 'item_cat_id_fix_cnt_month_mean_Expanding_lag_12',
 'item_id_cnt_month_mean_Expanding_lag_12',
 'item_id',
 'item_cnt_month_lag_4',
 'item_category_id_cnt_month_mean_Expanding_lag_2',
 'shop_id_cnt_month_mean_Expanding_lag_3',
 'days_of_month',
 'shop_id_cnt_month_mean_Expanding_lag_

In [135]:
# # Now fill `X_train_level2` with metafeatures
slice_start = 0

for cur_block_num in tqdm(months_to_generate_meta_features):

    print('-' * 50)
    print('Start training for month%d'% cur_block_num)
    start_cur_month = time.perf_counter()

    cur_X_train = all_data.loc[dates <  cur_block_num][feature_columns]
    cur_X_test =  all_data.loc[dates == cur_block_num][feature_columns]

    cur_y_train = all_data.loc[dates <  cur_block_num, Target].values
    cur_y_test =  all_data.loc[dates == cur_block_num, Target].values

    # Create Numpy arrays of train, test and target dataframes to feed into models
    train_x = cur_X_train.values
    train_y = cur_y_train.ravel()
    test_x = cur_X_test.values
    test_y = cur_y_test.ravel()

    preds = []

    from sklearn.linear_model import (LinearRegression, SGDRegressor)
    import lightgbm as lgb

    sgdr= SGDRegressor(
        penalty = 'l2' ,
        random_state = SEED )
    lgb_params = {
                  'feature_fraction': 0.75,
                  'metric': 'rmse',
                  'nthread':1,
                  'min_data_in_leaf': 2**7,
                  'bagging_fraction': 0.75,
                  'learning_rate': 0.03,
                  'objective': 'mse',
                  'bagging_seed': 2**7,
                  'num_leaves': 2**7,
                  'bagging_freq':1,
                  'verbose':0
                  }

    estimators = [sgdr]

    for estimator in estimators:
        print('Training Model %d: %s'%(len(preds), estimator.__class__.__name__))
        start = time.perf_counter()
        estimator.fit(train_x, train_y)
        pred_test = estimator.predict(test_x)
        preds.append(pred_test)
        # pred_train = estimator.predict(train_x)
        # print('Train RMSE for %s is %f' % (estimator.__class__.__name__, sqrt(mean_squared_error(cur_y_train, pred_train))))
        # print('Test RMSE for %s is %f' % (estimator.__class__.__name__, sqrt(mean_squared_error(cur_y_test, pred_test))))
        run = time.perf_counter() - start
        print('{} runs for {:.2f} seconds.'.format(estimator.__class__.__name__, run))
        print()


    print('Training Model %d: %s'%(len(preds), 'lightgbm'))
    start = time.perf_counter()
    estimator = lgb.train(lgb_params, lgb.Dataset(train_x, label=train_y), 300)
    pred_test = estimator.predict(test_x)
    preds.append(pred_test)
    # pred_train = estimator.predict(train_x)
    # print('Train RMSE for %s is %f' % ('lightgbm', sqrt(mean_squared_error(cur_y_train, pred_train))))
    # print('Test RMSE for %s is %f' % ('lightgbm', sqrt(mean_squared_error(cur_y_test, pred_test))))
    run = time.perf_counter() - start
    print('{} runs for {:.2f} seconds.'.format('lightgbm', run))
    print()


    print('Training Model %d: %s'%(len(preds), 'keras'))
    start = time.perf_counter()
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.wrappers.scikit_learn import KerasRegressor

    def baseline_model():
    	# create model
        model = Sequential()
        model.add(Dense(20, input_dim=train_x.shape[1], kernel_initializer='uniform', activation='softplus'))
        model.add(Dense(1, kernel_initializer='uniform', activation = 'relu'))
        # Compile model
        model.compile(loss='mse', optimizer='Nadam', metrics=['mse'])
        # model.compile(loss='mean_squared_error', optimizer='adam')
        return model

    estimator = KerasRegressor(build_fn=baseline_model, verbose=1, epochs=5, batch_size = 55000)

    estimator.fit(train_x, train_y)
    pred_test = estimator.predict(test_x)
    preds.append(pred_test)

    run = time.perf_counter() - start
    print('{} runs for {:.2f} seconds.'.format('lightgbm', run))


    cur_month_run_total = time.perf_counter() - start_cur_month
    print('Total running time was {:.2f} minutes.'.format(cur_month_run_total/60))
    print('-' * 50)

    slice_end = slice_start + cur_X_test.shape[0]
    X_all_level2[ slice_start : slice_end , :] = np.c_[preds].transpose()
    slice_start = slice_end



  0%|          | 0/8 [00:00<?, ?it/s]

--------------------------------------------------
Start training for month27
Training Model 0: SGDRegressor


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor runs for 9.27 seconds.

Training Model 1: lightgbm
lightgbm runs for 232.59 seconds.

Training Model 2: keras


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


...

lightgbm runs for 41.75 seconds.
Total running time was 5.97 minutes.
--------------------------------------------------
--------------------------------------------------
Start training for month33
Training Model 0: SGDRegressor
SGDRegressor runs for 11.36 seconds.

Training Model 1: lightgbm
lightgbm runs for 299.11 seconds.

Training Model 2: keras
Epoch 1/5
5023308/5023308 [==============================] - 13s 3us/step - loss: 1.3125 - mean_squared_error: 1.3125



 88%|████████▊ | 7/8 [37:15<05:19, 319.40s/it]

lightgbm runs for 60.12 seconds.
Total running time was 6.24 minutes.
--------------------------------------------------
--------------------------------------------------
Start training for month34
Training Model 0: SGDRegressor
SGDRegressor runs for 12.33 seconds.

Training Model 1: lightgbm
lightgbm runs for 325.11 seconds.

Training Model 2: keras
Epoch 1/5
5245110/5245110 [==============================] - 12s 2us/step - loss: 1.2820 - mean_squared_error: 1.2820



100%|██████████| 8/8 [43:49<00:00, 328.69s/it]



lightgbm runs for 53.45 seconds.
Total running time was 6.56 minutes.
--------------------------------------------------


In [136]:
# # Split train and test
test_nrow = len(preds[0])
X_train_level2 = X_all_level2[ : -test_nrow, :]
X_test_level2 = X_all_level2[ -test_nrow: , :]
y_train_level2 = y_all_level2[ : -test_nrow]
y_test_level2 = y_all_level2[ -test_nrow : ]

# print('%0.2f min: Finish training First level models'%((time.perf_counter() - start_first_level_total)/60))

In [138]:
# # 4. Ensembling -------------------------------------------------------------------
pred_list = {}

# # A. Second level learning model via linear regression
print('Training Second level learning model via linear regression')

from sklearn.linear_model import (LinearRegression, SGDRegressor)
lr = LinearRegression()
lr.fit(X_train_level2, y_train_level2)
# # Compute R-squared on the train and test sets.
# # print('Train R-squared for %s is %f' %('test_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, lr.predict(X_train_level2)))))
test_preds_lr_stacking = lr.predict(X_test_level2)
train_preds_lr_stacking = lr.predict(X_train_level2)
print('Train R-squared for %s is %f' %('train_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, train_preds_lr_stacking))))

pred_list['test_preds_lr_stacking'] = test_preds_lr_stacking
if Validation:
    print('Test R-squared for %s is %f' %('test_preds_lr_stacking', sqrt(mean_squared_error(y_test_level2, test_preds_lr_stacking))))


# # B. Second level learning model via SGDRegressor
print('Training Second level learning model via SGDRegressor')
sgdr= SGDRegressor(
    penalty = 'l2' ,
    random_state = SEED )

sgdr.fit(X_train_level2, y_train_level2)
# # Compute R-squared on the train and test sets.
# # print('Train R-squared for %s is %f' %('test_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, lr.predict(X_train_level2)))))
test_preds_sgdr_stacking = sgdr.predict(X_test_level2)
train_preds_sgdr_stacking = sgdr.predict(X_train_level2)
print('Train R-squared for %s is %f' %('train_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, train_preds_sgdr_stacking))))

pred_list['test_preds_sgdr_stacking'] = test_preds_sgdr_stacking
if Validation:
    print('Test R-squared for %s is %f' %('test_preds_sgdr_stacking', sqrt(mean_squared_error(y_test_level2, test_preds_sgdr_stacking))))


# print('%0.2f min: Finish training second level model'%((time.time() - start_time)/60))

Training Second level learning model via linear regression
Train R-squared for train_preds_lr_stacking is 0.818117
Training Second level learning model via SGDRegressor


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Train R-squared for train_preds_lr_stacking is 0.821289


In [139]:
# # Submission -------------------------------------------------------------------
if not Validation:
    submission = pd.read_csv('%s/sample_submission.csv' % data_path)

    ver = 6
    for pred_ver in ['lr_stacking', 'sgdr_stacking']:
        print(pred_list['test_preds_' + pred_ver].clip(0,20).mean())
        submission['item_cnt_month'] = pred_list['test_preds_' + pred_ver].clip(0,20)
        submission[['ID', 'item_cnt_month']].to_csv('%s/ver%d_%s.csv' % (submission_path, ver, pred_ver), index = False)

# print('%0.2f min: Finish running scripts'%((time.time() - start_time)/60))

NameError: name 'data_path' is not defined